In [47]:
import numpy as np
import librosa
import torch
import CLAP.src.laion_clap as laion_clap
from sklearn.metrics.pairwise import cosine_similarity

def int16_to_float32(x):
    return (x / 32767.0).astype(np.float32)

def float32_to_int16(x):
    x = np.clip(x, a_min=-1., a_max=1.)
    return (x * 32767.).astype(np.int16)

def get_cosine_similarity(vector1, vector2):
    similarity = cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))
    return similarity[0, 0]

def euclidean_distance(tensor1, tensor2):
    distance = np.linalg.norm(tensor1 - tensor2)
    return distance

In [2]:
model = laion_clap.CLAP_Module(enable_fusion=False, amodel= 'HTSAT-base', device='cuda')

state_dict = torch.load("/workspace/clap_final_0520_augmentation_epoch_best_29.pth")
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith("_orig_mod."):
        new_key = k.replace("_orig_mod.", "")
    else:
        new_key = k
    new_state_dict[new_key] = v
model.load_state_dict(new_state_dict)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassificat

<All keys matched successfully>

In [20]:
import pandas as pd
from tqdm import tqdm
import os

aps = os.listdir("outputs")
df = pd.read_csv("/workspace/labeled_ss_sfx.csv")

generateds = []
for ap in tqdm(aps):
    ap = '_'.join(ap.split("_")[3:])
    data = df[df['audio_path'].str.contains(ap[:-6])].iloc[0]
    nap = data['audio_path']
    nc = data['new_caption']
    generateds.append({
        'audio_path': nap,
        'caption': nc
    })
pd.DataFrame(generateds).to_csv("gens.csv")

  0%|          | 4/52482 [00:00<15:13, 57.46it/s]


IndexError: single positional indexer is out-of-bounds

In [15]:
import sys
import pandas as pd

# 출력 길이 제한 늘리기
pd.set_option('display.max_colwidth', None)

In [23]:
df = pd.read_csv('running.csv')
df

,Unnamed: 0,audio_path,cosiine_similarity,new_caption,caption,duration
0,0,./outputs/music_loops_page_158_85880-Lil_Wayne_21-90bpm_0.wav,0,"A 90 bpm hip hop beat played on electric drums, mostly on the snare and hit-hats.",85880 Lil Wayne 21 90bpm,10.710204
1,1,./outputs/music_loops_page_158_85880-Lil_Wayne_21-90bpm_1.wav,0,"A 90 bpm hip hop beat played on electric drums, mostly on the snare and hit-hats.",85880 Lil Wayne 21 90bpm,10.710204
2,2,./outputs/comic_film_fx_page_1274_541327-zapsplat-foley-wooden-box-case-set-down-gently-hard-surface-005-61780_0.wav,0,A small wooden box being set on wooden floor planks.,541327 zapsplat foley wooden box case set down gently hard surface 005 61780,0.595238
3,3,./outputs/comic_film_fx_page_1274_541327-zapsplat-foley-wooden-box-case-set-down-gently-hard-surface-005-61780_1.wav,0,A small wooden box being set on wooden floor planks.,541327 zapsplat foley wooden box case set down gently hard surface 005 61780,0.595238
4,4,./outputs/multimedia_page_658_253013-SW001-8-Bit-Games-356-3xSound-Combinations-Wacky_0.wav,0,An 8-bit high-pitched swelling static sound effect.,253013 SW001 8 Bit Games 356 3xSound Combinations Wacky,2.246531
...,...,...,...,...,...,...
19995,19995,./outputs/comic_film_fx_page_392_445263-125-Hydrophone-Underwater-Mono-96000-24_1.wav,0,Splashing and bubbling sound when water is moving underwater,445263 125 Hydrophone Underwater Mono 96000 24,5.000000
19996,19996,./outputs/human_page_1147_67698-Young_male_trucker_interior_cab_cb_radio_voice_says_that-s_a_big_10-4-BLASTWAVEFX-05463_0.wav,0,"A male voice over a radio says, ""That's a big 10 4""",67698 Young male trucker interior cab cb radio voice says that s a big 10 4 BLASTWAVEFX 05463,1.933061
19997,19997,./outputs/human_page_1147_67698-Young_male_trucker_interior_cab_cb_radio_voice_says_that-s_a_big_10-4-BLASTWAVEFX-05463_1.wav,0,"A male voice over a radio says, ""That's a big 10 4""",67698 Young male trucker interior cab cb radio voice says that s a big 10 4 BLASTWAVEFX 05463,1.933061
19998,19998,./outputs/multimedia_page_882_505890-SBsfi2-Bleep_Select_002_0.wav,0,A high-pitched beeping sound effect when a device finishes scanning an object,505890 SBsfi2 Bleep Select 002,0.948571


In [52]:
import torchaudio

model.eval()

total = 0
claps = []
for i in tqdm(range(len(df))):
    data = df.iloc[i]
    caption = data['new_caption']
    
    text_embed = model.get_text_embedding([caption])
    all_embeds = [
        torchaudio.load(data['audio_path']),
    ]

    audio_embed = model.get_audio_embedding_from_data(x = [np.array(all_embeds[0][0][0])])
    simil = get_cosine_similarity(audio_embed, text_embed)
    total += simil.item()
    claps.append({
        'audio_path': data['audio_path'],
        'cosiine_similarity': simil.item(),
        'new_caption': caption,
        'caption': data['caption'],
    })

pd.DataFrame(claps).to_csv("get_cosines.csv")
print(total/len(df))

100%|██████████| 20000/20000 [41:44<00:00,  7.99it/s] 


0.2221825357604539


In [1]:
from msclap import CLAP
from torch.nn import functional as F
import torch

clap = CLAP(version="2023", use_cuda=torch.cuda.is_available())

In [2]:
@torch.no_grad()
@torch.autocast(device_type="cuda", enabled=False)
def clap_rank(audios, texts):
    # audios = audios.mean(dim=-1)
    audios = audios.float()
    text_embed = clap.get_text_embeddings(texts)
    audio_embed = clap.clap.audio_encoder(audios)[0]

    similarity = F.cosine_similarity(text_embed, audio_embed)
    args = torch.argsort(similarity, dim=0, descending=True)
    return similarity

In [3]:
import torchaudio
from tqdm import tqdm
import pandas as pd

df = pd.read_csv("running.csv")

total = 0
claps = []
for i in tqdm(range(len(df))):
    data = df.iloc[i]
    caption = data['new_caption']
    
    audio, sr = torchaudio.load(data['audio_path'])
    audio = audio / audio.abs().amax(dim=1, keepdim=True).clamp_min(1)

    simil = clap_rank(audio[0].unsqueeze(dim=0).to('cuda'), [caption])
    total += simil.item()
    claps.append({
        'audio_path': data['audio_path'],
        'cosiine_similarity': simil.item(),
        'new_caption': caption,
        'caption': data['caption'],
    })

pd.DataFrame(claps).to_csv("get_cosines_ms.csv")
print(total/len(df))

100%|██████████| 20000/20000 [50:36<00:00,  6.59it/s] 


0.36061684184131443


In [6]:
import pandas as pd
import torchaudio

df = pd.read_csv("running.csv")
df.iloc[0]['audio_path']

'./outputs/music_loops_page_158_85880-Lil_Wayne_21-90bpm_0.wav'

In [7]:
audio, sr = torchaudio.load('./outputs/music_loops_page_158_85880-Lil_Wayne_21-90bpm_0.wav')

In [8]:
audio = audio / audio.abs().amax(dim=1, keepdim=True).clamp_min(1)

In [18]:
audio[0].unsqueeze(dim=0).unsqueeze(dim=0).shape

torch.Size([1, 1, 441000])

In [38]:
sim = clap_rank(audio[0].unsqueeze(dim=0).to('cuda'), ['test'])

torch.Size([1, 441000])
torch.Size([1, 441000])


In [ ]:
sim

In [4]:
df1 = pd.read_csv("get_cosines.csv")
df2 = pd.read_csv("get_cosines_ms.csv")

In [7]:
df_sorted1 = df1.sort_values(by="cosiine_similarity")
df_sorted2 = df2.sort_values(by="cosiine_similarity")

In [19]:
from audiotools import AudioSignal

for i in range(10):
    print(df_sorted2.iloc[i]['new_caption'], df_sorted2.iloc[i]['cosiine_similarity'])
    # AudioSignal(df_sorted2.iloc[i]['audio_path']).widget()
    print("\n")

for i in range(1, 10):
    print(df_sorted2.iloc[-i]['new_caption'], df_sorted2.iloc[-i]['cosiine_similarity'])
    # AudioSignal(df_sorted2.iloc[-i]['audio_path']).widget()
    print("\n")

The sound of a woman coughing twice -0.1708285957574844


The sound of a person snorting with a sniffle -0.1588117182254791


The sound of a person snorting with a sniffle -0.1569676697254181


The scraping sound of scratching a wooden surface -0.1410888731479644


The heavy breathing sound of someone using a gas mask -0.1369206011295318


The heavy breathing sound of someone using a gas mask -0.1306816935539245


The sound of a woman coughing twice -0.1304907500743866


Hammering a wooden plank once -0.1265505254268646


The scraping sound of scratching a wooden surface -0.1206145212054252


Rustling sound of handling a tactical backpack -0.1180179491639137


A sound effect of a high-pitched, metallic clank sound of two swords striking each other, with a delay and decay effect. 0.713029146194458


Someone spraying short squeaky bursts from an air freshener can multiple times. 0.7121791243553162


An ascending high-pitched, high-frequency sound effect of insects chirping used in a sci-

In [18]:
from audiotools import AudioSignal

for i in range(1, 10):
    print(df_sorted1.iloc[i]['new_caption'], df_sorted1.iloc[i]['cosiine_similarity'])
    # AudioSignal(df_sorted1.iloc[i]['audio_path']).widget()
    print("\n")
for i in range(1, 10):
    print(df_sorted1.iloc[-i]['new_caption'], df_sorted1.iloc[-i]['cosiine_similarity'])
    # AudioSignal(df_sorted1.iloc[-i]['audio_path']).widget()
    print("\n")

A vibrating soft, light synth sound effect. -0.2051395028829574


The sound of an arrow being loaded into a wooden bow. -0.1994385570287704


Wood drawer with rusted iron rollers pulls out quickly -0.1960050612688064


Someone drops a small stone into a bucket of water, creating a splashing noise. -0.1794668138027191


The whirring sound of a small eco-friendly motorcycle passes by. -0.1772372275590896


A robotic plane beeping and flying by -0.1744576692581176


Sound effect in game, rocket fast whoosh -0.1719337105751037


unplayable -0.1668865978717804


The sound of a machine cover opening and sliding. -0.1664419770240783


Scraping sound when pushing a metal cabinet on a concrete surface 0.6321414709091187


The whirring sound of opening a drawer filled with silverware, as it makes a thump when the drawer is fully opened and the silverware clatters, twice 0.6281040906906128


Someone playing a single reverberated note on a piano, given a fantasy and heavenly sound effect. 0.626240

In [16]:
sum(df_sorted1['cosiine_similarity'].to_list())

4443.650715209078

In [17]:
sum(df_sorted2['cosiine_similarity'].to_list())

7212.336836826289